In [1]:
import pandas as pd


In [2]:
df = pd.read_csv("model1/data/train.csv")

In [13]:
text = df['text'].values

In [14]:
words = [i.split() for i in text]

In [15]:
lens = [len(i) for i in words]

In [17]:
sum(lens) / len(lens)

12.840175074867542

In [18]:
max(lens)

33

In [3]:
import transformers
from transformers import AutoModel
from transformers import AutoTokenizer
import torchsummary
from torchsummary import summary


In [4]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [18]:
tokens = tokenizer("nervous")['input_ids']

In [19]:
tokenizer.convert_ids_to_tokens(tokens)

['[CLS]', 'nervous', '[SEP]']

In [11]:
roberta = AutoModel.from_pretrained("roberta-base")



Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
summary(roberta, depth=5)

Layer (type:depth-idx)                                  Param #
├─RobertaEmbeddings: 1-1                                --
|    └─Embedding: 2-1                                   38,603,520
|    └─Embedding: 2-2                                   394,752
|    └─Embedding: 2-3                                   768
|    └─LayerNorm: 2-4                                   1,536
|    └─Dropout: 2-5                                     --
├─RobertaEncoder: 1-2                                   --
|    └─ModuleList: 2-6                                  --
|    |    └─RobertaLayer: 3-1                           --
|    |    |    └─RobertaAttention: 4-1                  --
|    |    |    |    └─RobertaSelfAttention: 5-1         1,771,776
|    |    |    |    └─RobertaSelfOutput: 5-2            592,128
|    |    |    └─RobertaIntermediate: 4-2               --
|    |    |    |    └─Linear: 5-3                       2,362,368
|    |    |    |    └─GELUActivation: 5-4               --
|    |    |    

Layer (type:depth-idx)                                  Param #
├─RobertaEmbeddings: 1-1                                --
|    └─Embedding: 2-1                                   38,603,520
|    └─Embedding: 2-2                                   394,752
|    └─Embedding: 2-3                                   768
|    └─LayerNorm: 2-4                                   1,536
|    └─Dropout: 2-5                                     --
├─RobertaEncoder: 1-2                                   --
|    └─ModuleList: 2-6                                  --
|    |    └─RobertaLayer: 3-1                           --
|    |    |    └─RobertaAttention: 4-1                  --
|    |    |    |    └─RobertaSelfAttention: 5-1         1,771,776
|    |    |    |    └─RobertaSelfOutput: 5-2            592,128
|    |    |    └─RobertaIntermediate: 4-2               --
|    |    |    |    └─Linear: 5-3                       2,362,368
|    |    |    |    └─GELUActivation: 5-4               --
|    |    |    

In [13]:
from transformers import BertTokenizer, AutoTokenizer


In [14]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base')


In [37]:
sentence = "hello i am alex"

In [52]:
tokenizer.encode_plus("test", "test2")

{'input_ids': [0, 21959, 2, 2, 21959, 176, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [53]:
tokenizer.decode(2)

'</s>'

In [39]:
tokens = tokenizer(sentence, return_tensors="pt")
print(tokens['input_ids'])

tensor([[    0, 42891,   939,   524,    10, 14726,     2]])


In [44]:
embeddings = roberta.embeddings(input_ids=tokens['input_ids'])
embeddings

tensor([[[ 0.1664, -0.0541, -0.0014,  ..., -0.0811,  0.0794,  0.0155],
         [-0.1150, -0.1756, -0.1607,  ..., -0.0596, -0.3592,  0.9941],
         [ 0.1324,  0.1079, -0.3674,  ...,  0.3495, -0.1283,  0.4034],
         ...,
         [-0.1279, -0.3089, -0.1310,  ...,  0.3103, -0.1026,  0.5261],
         [-0.3172, -0.0767,  0.2338,  ...,  0.1195,  0.4922,  0.3705],
         [ 0.2360, -0.2293, -0.0167,  ...,  0.4182,  0.1640,  0.1783]]],
       grad_fn=<NativeLayerNormBackward0>)

In [43]:
embeddings.shape

torch.Size([1, 7, 768])

In [47]:
logits = roberta.encoder(embeddings)

In [49]:
logits.last_hidden_state.shape

torch.Size([1, 7, 768])

In [ ]:
"""

original text:
hi how are you

tokenizer (tokens):
45 120 23 50

MODEL:

Embeddings:
[768] [768] [768] [768]

Encoder:




"""

In [12]:
roberta

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou